In [7]:
import praw
import pandas as pd
import datetime
import re

In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
import os
REDDIT_CLIENT_ID = os.getenv('REDDIT_CLIENT_ID')
REDDIT_CLIENT_SECRET = os.getenv('REDDIT_CLIENT_SECRET')
REDDIT_USER_AGENT = os.getenv('REDDIT_USER_AGENT')

In [5]:
reddit = praw.Reddit(
    client_id='zYdDXB3Ntg3zWzGSZVwNCQ',
    client_secret='JNU1Wx-n9Xy9tZfviPsb_gSA0lB-Gg',
    user_agent='script:reddit_stock_scraper:v1.0 (by /u/AmoebaOld7828)'
)

try:
    print("Authenticated as:", reddit.user.me())
except Exception as e:
    print("Failed to connect:", e)

Authenticated as: None


In [6]:
subreddits = ['investing', 'wallstreetbets', 'stocks']
keywords = ['TSLA', 'AAPL', 'NVDA', 'AMZN', 'MSFT']

for subreddit_name in subreddits:
    subreddit = reddit.subreddit(subreddit_name)
    for post in subreddit.new(limit=100):
        title = post.title
        if any(keyword in title.upper() for keyword in keywords):
            print(f"{subreddit_name}: {title} (Score: {post.score}, Comments: {post.num_comments})")

wallstreetbets: Tsla puts yolo (Score: 78, Comments: 58)
wallstreetbets: Early 2020 - TSLA Rally (Score: 9, Comments: 15)
wallstreetbets: Saw HIMS surge and said fuck it puts it is and a NVDA bag too (Score: 34, Comments: 14)
wallstreetbets: TSLA target 400$ (Score: 0, Comments: 57)
wallstreetbets: £20k in one month NVDA calls (Score: 44, Comments: 6)
wallstreetbets: YOLO nvda calls (Score: 145, Comments: 155)


In [8]:
# CONFIGURATION
REDDIT_CLIENT_ID = REDDIT_CLIENT_ID
REDDIT_CLIENT_SECRET = REDDIT_CLIENT_SECRET
REDDIT_USER_AGENT = REDDIT_USER_AGENT
SUBREDDITS = ['investing', 'wallstreetbets', 'stocks']
TICKERS = ['TSLA', 'AAPL', 'MSFT', 'NVDA', 'AMZN']
POST_LIMIT = 1000  # per subreddit
OUTPUT_FILE = 'reddit_posts.csv'
# -----------------------------------

reddit = praw.Reddit(client_id=REDDIT_CLIENT_ID,
                     client_secret=REDDIT_CLIENT_SECRET,
                     user_agent=REDDIT_USER_AGENT)

def extract_tickers(text):
    found = []
    for ticker in TICKERS:
        if re.search(r'\b' + re.escape(ticker) + r'\b', text.upper()):
            found.append(ticker)
    return found

# data in list
all_posts = []

for sub_name in SUBREDDITS:
    print(f"Scraping r/{sub_name}...")
    subreddit = reddit.subreddit(sub_name)
    for post in subreddit.new(limit=POST_LIMIT):
        combined_text = (post.title + ' ' + (post.selftext or '')).upper()
        tickers_found = extract_tickers(combined_text)
        if not tickers_found:
            continue  # Skip if no relevant ticker

        all_posts.append({
            'title': post.title,
            'selftext': post.selftext,
            'author': str(post.author),
            'created_utc': datetime.datetime.utcfromtimestamp(post.created_utc),
            'score': post.score,
            'subreddit': sub_name,
            'tickers_mentioned': ','.join(tickers_found)
        })

Scraping r/investing...


/var/folders/vr/nc9jvld12q115x_g_114k8g40000gn/T/ipykernel_13353/3263431535.py:38: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  'created_utc': datetime.datetime.utcfromtimestamp(post.created_utc),


Scraping r/wallstreetbets...
Scraping r/stocks...


In [16]:
# Convert to DataFrame
df = pd.DataFrame(all_posts)
df.to_csv(OUTPUT_FILE, index=False)
print(f"\n Saved {len(df)} posts to {OUTPUT_FILE}")


 Saved 182 posts to reddit_posts.csv


In [17]:
data = pd.read_csv("reddit_posts.csv")

In [18]:
data

,title,selftext,author,created_utc,score,subreddit,tickers_mentioned
0,"Sold Hood and started positions in Apple, Goog...",Recently I sold all my Hood shares (I got 4300...,coopermug,2025-05-30 01:27:26,0,investing,NVDA
1,Semis not recovering despite new wave of AI sp...,Looking at how SMH did recently I can't help t...,SurveyIllustrious738,2025-05-29 07:45:20,6,investing,NVDA
2,NVIDIA Earnings Tomorrow – The Market’s Locomo...,"NVIDIA is reporting earnings tomorrow, and hon...",ZeroWallStreet,2025-05-27 15:15:46,43,investing,NVDA
3,TSLA: Why I'm still bullish despite the EPS no...,Been holding TSLA since 2021 and just watched ...,faharxpg,2025-05-27 09:58:01,122,investing,TSLA
4,I've got called old fashion for investing in i...,My mate told me I'm old fashioned for investin...,moodyhighway,2025-05-26 17:16:57,26,investing,NVDA
...,...,...,...,...,...,...,...
177,Waiting for the day AMZN and GOOGL erupt so I ...,I’ll wait two years for them to be in their 30...,Virtual-Tonight-2444,2025-05-01 18:24:39,0,stocks,AMZN
178,(05/1) Interesting Stocks Today - Unemployment...,Hi! I am an ex-prop shop equity trader. This i...,WinningWatchlist,2025-05-01 13:08:57,13,stocks,"TSLA,AAPL,MSFT,NVDA,AMZN"
179,"Inheriting a small fortune, considering how to...","This is probably an extremely tired topic, but...",ishouldgooutmore,2025-05-01 06:38:50,1,stocks,"MSFT,NVDA,AMZN"
180,How can I lock in profits on a short put as th...,I sold a put option on MSFT with a $435 strike...,Bright_Office_9792,2025-04-30 22:23:05,5,stocks,MSFT
